In [ ]:
!gdown --id 1-AlW7oNJHaqi3xk_9dWHUS52Dzl_FmFW

In [ ]:
!gdown --id 1-8TsrqTRFP-q9TM-6HinhO0ZVXFHq9TB

In [31]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [32]:
import pandas as pd
import numpy as np

In [33]:
import os

# Define the folder path
folder_path = '/content/drive/MyDrive/bootcamp/data'

# Create the folder if it doesn't already exist
os.makedirs(folder_path, exist_ok=True)

In [ ]:
import shutil

# Move train_data.csv
shutil.move('/content/train_data.csv', f'{folder_path}/train_data.csv')

# Move test_data.csv
shutil.move('/content/test_data.csv', f'{folder_path}/test_data.csv')

In [34]:
train_data_path = f'{folder_path}/train_data.csv'
test_data_path = f'{folder_path}/test_data.csv'

train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)

<ipython-input-34-e081c316124a>:4: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv(train_data_path)


In [35]:
train_data.head()

,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime
0,2,NaN,False,2016-11-11,A2OSUEZJIN7BI,0511189877,NaN,Chris,I have an older URC-WR7 remote and thought thi...,Cannot Learn,1478822400
1,5,NaN,True,2016-06-06,A2NETQRG6JHIG7,0511189877,NaN,Qrysta White,First time I've EVER had a remote that needed ...,zero programming needed! Miracle!?,1465171200
2,4,NaN,True,2016-03-10,A12JHGROAX49G7,0511189877,NaN,Linwood,Got them and only 2 of them worked. company ca...,Works Good and programs easy.,1457568000
3,5,NaN,True,2016-01-14,A1KV65E2TMMG6F,0511189877,NaN,Dane Williams,I got tired of the remote being on the wrong s...,Same as TWC remote,1452729600
4,5,NaN,True,2016-10-20,A280POPEWI0NSA,0594459451,NaN,Kristina H.,After purchasing cheap cords from another webs...,Good Quality Cord,1476921600


In [36]:
train_data.shape

(838944, 11)

In [37]:
train_data['reviewText'][0]

"I have an older URC-WR7 remote and thought this would be an upgrade (and because TWC stuck me with it), but this one fails where the other one didn't.  The old remote could go head to head and learn.  I have 2 different Blu Ray players (LG & Panasonic) and this one fails on both.  It cannot learn buttons.  The biggest problem is in streaming when I need to hit the blue, red, yellow, green buttons.  These cannot work either of them, so I have to pull out the remote for that one setting.  I don't know why they give you multiple code methods but no learning."

In [38]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 838944 entries, 0 to 838943
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   overall         838944 non-null  int64 
 1   vote            191468 non-null  object
 2   verified        838944 non-null  bool  
 3   reviewTime      838944 non-null  object
 4   reviewerID      838944 non-null  object
 5   asin            838944 non-null  object
 6   style           490613 non-null  object
 7   reviewerName    838717 non-null  object
 8   reviewText      838944 non-null  object
 9   summary         838868 non-null  object
 10  unixReviewTime  838944 non-null  int64 
dtypes: bool(1), int64(2), object(8)
memory usage: 64.8+ MB


In [39]:
train_data['reviewText'] = train_data['reviewText'].str.lower().str.strip()
train_data['summary'] = train_data['summary'].str.lower().str.strip()

test_data['reviewText'] = test_data['reviewText'].str.lower().str.strip()
test_data['summary'] = test_data['summary'].str.lower().str.strip()

In [40]:
import re

# Define a regex pattern that removes all punctuation except for ?! and .
pattern = r'[^a-zA-Z0-9\s?!\.]'

# Apply this to the reviewText and summary columns, with a check for string type
train_data['reviewText'] = train_data['reviewText'].apply(lambda x: re.sub(pattern, '', x) if isinstance(x, str) else x)
train_data['summary'] = train_data['summary'].apply(lambda x: re.sub(pattern, '', x) if isinstance(x, str) else x)

# Assuming you have a similar structure for test_data
test_data['reviewText'] = test_data['reviewText'].apply(lambda x: re.sub(pattern, '', x) if isinstance(x, str) else x)
test_data['summary'] = test_data['summary'].apply(lambda x: re.sub(pattern, '', x) if isinstance(x, str) else x)

In [41]:
!pip install textblob

In [ ]:
from textblob import TextBlob

def correct_spelling(text):
    return str(TextBlob(text).correct())

train_data['reviewText'] = train_data['reviewText'].apply(correct_spelling)
train_data['summary'] = train_data['summary'].apply(correct_spelling)

test_data['reviewText'] = test_data['reviewText'].apply(correct_spelling)
test_data['summary'] = test_data['summary'].apply(correct_spelling)

In [42]:
!pip install symspellpy

In [43]:
import pkg_resources
from symspellpy import SymSpell, Verbosity

In [44]:
dictionary_path = pkg_resources.resource_filename("symspellpy", "frequency_dictionary_en_82_765.txt")

sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

input_term = "memebers"

suggestions = sym_spell.lookup(input_term, Verbosity.CLOSEST, max_edit_distance=2)
for suggestion in suggestions:
    print(suggestion)

members, 1, 226656153


In [45]:
from operator import attrgetter

dictionary_cache = []


def find_nearest_word_in_dict(word):
    word = word.lower()

    if word in dictionary_cache:
        return word

    original_word = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=0)

    if original_word:
        dictionary_cache.append(original_word[0].term)
        return original_word[0].term
    else:
        suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=2, include_unknown=True)
        suggestions = sorted(suggestions, key=attrgetter('distance'))

        min_distance = suggestions[0].distance
        nearest_word = max([s for s in suggestions if s.distance == min_distance], key=attrgetter('count'))

        dictionary_cache.append(nearest_word.term)

        return nearest_word.term

In [46]:
sample_sentence = train_data['reviewText'][9]
sample_sentence

'brand name adapter this is just like the one you got when you bought your nook. i would always recommend using a designated adapter. the charging cable is a whole different story but the adapter defiantly use the designated charger.'

In [47]:
tokens = sample_sentence.split()
tokens

['brand',
 'name',
 'adapter',
 'this',
 'is',
 'just',
 'like',
 'the',
 'one',
 'you',
 'got',
 'when',
 'you',
 'bought',
 'your',
 'nook.',
 'i',
 'would',
 'always',
 'recommend',
 'using',
 'a',
 'designated',
 'adapter.',
 'the',
 'charging',
 'cable',
 'is',
 'a',
 'whole',
 'different',
 'story',
 'but',
 'the',
 'adapter',
 'defiantly',
 'use',
 'the',
 'designated',
 'charger.']

In [48]:
corrected_tokens = []

for t in tokens:
    nearest = find_nearest_word_in_dict(t)
    corrected_tokens.append(nearest)

### Mis-spelling correction example

In [49]:
for t, c in zip(tokens, corrected_tokens):
    print(t, c)

brand brand
name name
adapter adapter
this this
is is
just just
like like
the the
one one
you you
got got
when when
you you
bought bought
your your
nook. nook
i i
would would
always always
recommend recommend
using using
a a
designated designated
adapter. adapter
the the
charging charging
cable cable
is is
a a
whole whole
different different
story story
but but
the the
adapter adapter
defiantly defiantly
use use
the the
designated designated
charger. charger


In [68]:
def correct_spelling_symspell(sentence):
    tokens = sentence.split()
    corrected_tokens = list(map(find_nearest_word_in_dict, tokens))

    return ' '.join(corrected_tokens)

In [69]:
corrected_sentence = correct_spelling_symspell(sample_sentence)

print('main sentence: {}'.format(sample_sentence))
print('corrected sentence: {}'.format(corrected_sentence))

main sentence: brand name adapter this is just like the one you got when you bought your nook. i would always recommend using a designated adapter. the charging cable is a whole different story but the adapter defiantly use the designated charger.
corrected sentence: brand name adapter this is just like the one you got when you bought your nook i would always recommend using a designated adapter the charging cable is a whole different story but the adapter defiantly use the designated charger


In [ ]:
dictionary_cache = []

for i in range(100):
    print(i)
    result = correct_spelling_symspell(train_data['reviewText'][i])